# Clock Drift

This notebook demonstates the functional of the clock drift transform. The transform models the sampling rate having some error, represented by an accumulate Gaussian random variable whose mean is zero and whose variance is according to the PPM error parameter. The effect will increase or decrease the sampling rate randomly over the course of the input signal. Over long sequences this will produce a handful of more or less samples than were input.

In [ ]:
import torchsig.transforms.functional as F

import numpy as np
import scipy as sp
%matplotlib inline
import matplotlib.pyplot as plt

A tone signal is generated to test the clock drift transform. The error is defined in parts-per-million (PPM). A larger PPM represents a larger frequency error on average. Using 0 PPM will result in no drift.

In [ ]:
# test functional
rng = np.random.default_rng(42)

N = 10000
sample_rate = 10e6
#center_frequency = 0
center_frequency = sample_rate/N
n = np.arange(0,N)
t = n/sample_rate
tone_bb_data = np.exp(2j*np.pi*center_frequency*t)

drift_ppm=1

data_out = F.clock_drift(
    data = tone_bb_data,
    drift_ppm = drift_ppm,
    rng = rng
)

Time-domain plots show how the clock drift effects the complex sinusoid input. For small PPM errors the effect will may not be visible in the time domain.

In [ ]:
fig = plt.figure(figsize=(10,6))
fig.subplots_adjust(hspace=0.5)
ax = fig.add_subplot(2,1,1)
ax.plot(t,np.real(tone_bb_data),label='Real, Input Tone')
ax.plot(t,np.imag(tone_bb_data),label='Imag, Input Tone')
ylim = np.max(np.abs(tone_bb_data))*1.1
ax.set_ylim([-ylim,ylim])
ax.set_xlim([t[0],t[-1]])
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.grid()
ax.legend(loc='upper right')

ax = fig.add_subplot(2,1,2)
ax.plot(t,np.real(data_out),label='Real, Tone with Clock Drift')
ax.plot(t,np.imag(data_out),label='Imag, Tone with Clock Drift')
ax.set_ylim([-ylim,ylim])
ax.set_xlim([t[0],t[-1]])
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.grid()
ax.legend(loc='upper right')

In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.datasets.dataset_metadata import DatasetMetadata
from torchsig.signals.builders import ConstellationSignalBuilder
def generate_qpsk_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband QPSK Signal.

        Args:
        num_iq_samples (int, optional): Length of sample. Defaults to 10.
        scale (int, optional): scale normalized signal data. Defaults to 1.0.

        Returns:
            signal: generated Signal 

    """
    sample_rate = 10e6
    md = DatasetMetadata(
        num_iq_samples_dataset = num_iq_samples,
        fft_size = 4,
        impairment_level = 0,
        sample_rate = sample_rate,
        num_signals_max = 1,
        num_signals_min = 1,
        num_signals_distribution = [1.0],
        snr_db_min = 100.0,
        snr_db_max = 100.0,
        signal_duration_min = 1.00*num_iq_samples/sample_rate,
        signal_duration_max = 1.00*num_iq_samples/sample_rate,
        signal_bandwidth_min = sample_rate/4,
        signal_bandwidth_max = sample_rate/4,
        signal_center_freq_min = 0.0,
        signal_center_freq_max = 0.0,         
        class_list = ['qpsk'],
        class_distribution = [1.0],
        seed = 42
    )

    builder = ConstellationSignalBuilder(
        dataset_metadata = md, 
        class_name = 'qpsk',
        seed = 42
    )
    signal = builder.build()

    # normalize, then scale data   
    signal.data = F.normalize(
        data = signal.data,
        norm_order = 2,
        flatten = False
    )
    signal.data = np.multiply(signal.data, scale)

    return signal.data

A QPSK signal is used to demonstrate the drift. A larger drift PPM value is used to make the effect more obvious.

In [ ]:
# test data
N = 128
qpsk_data = generate_qpsk_signal(num_iq_samples = N)

# phase noise
impaired_qpsk_data = F.clock_drift(
    data = qpsk_data,
    drift_ppm=drift_ppm*100
)

n = np.arange(0,len(qpsk_data))
t = n/sample_rate

fig = plt.figure(figsize=(10,6))
fig.subplots_adjust(hspace=0.5)
ax = fig.add_subplot(2,1,1)
ax.plot(t,np.real(qpsk_data),label='Real, Input QPSK')
ax.plot(t,np.real(impaired_qpsk_data),label='Real, QPSK with Drift')
ylim = np.max(np.abs(qpsk_data))*1.1
#ax.set_ylim([-ylim,ylim])
ax.set_xlim([t[0],t[-1]])
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.grid()
ax.legend(loc='upper right')

ax = fig.add_subplot(2,1,2)
ax.plot(t,np.imag(qpsk_data),label='Imag, Input QPSK')
ax.plot(t,np.imag(impaired_qpsk_data),label='Imag, QPSK with Drift')
ax.set_ylim([-ylim,ylim])
ax.set_xlim([t[0],t[-1]])
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.grid()
ax.legend(loc='upper right')